In [ ]:
#Apertura dei dati da Google Drive
#Caricamento dei file testuali per navigare nelle cartelle e prendere le immagini
file=open("./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/train.txt", "r")
content = file.read()
train_set = content.strip().split(",")
print(train_set)
file.close()

file=open("./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/test.txt", "r")
content = file.read()
test_set = content.strip().split(",")
print(test_set)
file.close()


file=open("./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/all.txt", "r")
content = file.read()
all_locs = content.strip().split(",")
print(all_locs)
file.close()

channels = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']

['aguasclaras', 'bercy', 'bordeaux', 'nantes', 'paris', 'rennes', 'saclay_e', 'abudhabi', 'cupertino', 'pisa', 'beihai', 'hongkong', 'beirut', 'mumbai']
['brasilia', 'montpellier', 'norcia', 'rio', 'saclay_w', 'valencia', 'dubai', 'lasvegas', 'milano', 'chongqing']
['aguasclaras', 'bercy', 'bordeaux', 'nantes', 'paris', 'rennes', 'saclay_e', 'abudhabi', 'cupertino', 'pisa', 'beihai', 'hongkong', 'beirut', 'mumbai', 'brasilia', 'montpellier', 'norcia', 'rio', 'saclay_w', 'valencia', 'dubai', 'lasvegas', 'milano', 'chongqing']


In [ ]:
from PIL import Image
img = Image.open('./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/abudhabi/imgs_1_rect/B01.tif')
import torch
import torchvision.transforms.functional as F
tensor=F.to_tensor(img)
print(tensor)

print(img)



tensor([[[2123, 2115, 2096,  ..., 2368, 2363, 2359],
         [2124, 2116, 2097,  ..., 2367, 2363, 2360],
         [2127, 2119, 2100,  ..., 2366, 2363, 2360],
         ...,
         [2255, 2254, 2250,  ..., 2415, 2431, 2440],
         [2264, 2262, 2258,  ..., 2407, 2423, 2433],
         [2264, 2262, 2258,  ..., 2407, 2423, 2433]]], dtype=torch.int16)
<PIL.TiffImagePlugin.TiffImageFile image mode=I;16 size=785x799 at 0x7F3CDFD69A20>


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

transform=transforms.Compose([
    # your code here
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

class testTransforms():
  def __call__(self,sample):
    img1 = transforms.ToTensor()(sample[0])
    img2 = transforms.ToTensor()(sample[1])
    gt = np.array(sample[2])-1

    print(img1,img2,gt)

    img1 = transforms.Normalize(0,1)(img1)
    img2 = transforms.Normalize(0,1)(img2)
    gt = torch.from_numpy(gt)

    print(gt)

    return img1,img2,gt




#Inizilizzazione del dataset
class OSCD_dataset(torch.utils.data.Dataset):
    def __init__(self, train=True, normalize=True, transform=transform):
        self.train=train
        self.transform=transform
        self.normalize=normalize
        #Carica i dati dai file
        self.data = self.load_data()

    def __len__(self):
        # Restituisci la lunghezza del dataset
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        img1_list=sample[0]
        img2_list=sample[1]
        gt=sample[2]

        for ch in range(len(img1_list)):
          img1_list[ch] = np.array(img1_list[ch].getdata())
          img2_list[ch] = np.array(img2_list[ch].getdata())
        

        if self.transform is not None:
          sample=self.transform(sample)

        return sample

    def load_data(self):
      #selezione del sottoinsieme richiesto
        if self.train:
          self.req_set=train_set
        else:
          self.req_set=test_set
        all_data = []  # Lista per memorizzare tutti i dati da tutti i file
        all_imgs = []
        for loc in self.req_set:
            print('loading:', loc)
            img1_list = []
            img2_list = []

            for channel in channels:
                #prendo le immagini di ogni canale e le metto in una lista
                ch = Image.open('./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/' + loc + '/imgs_1_rect/' + channel + '.tif')
                img1_list.append(np.array(ch.getdata()).reshape(ch.size[0],ch.size[1]))
                ch = Image.open('./drive/MyDrive/OSCD_dataset/OSCD_dataset_images/' + loc + '/imgs_2_rect/' + channel + '.tif')
                img2_list.append(np.array(ch.getdata()).reshape(ch.size[0],ch.size[1]))    
            if self.train:
                #carico l'immagine della ground truth
                gt = Image.open('./drive/MyDrive/OSCD_dataset/OSCD_dataset_train_labels/' + loc + '/cm/' + loc + '-cm.tif')
            else:
                gt = Image.open('./drive/MyDrive/OSCD_dataset/OSCD_dataset_test_labels/' + loc + '/cm/' + loc + '-cm.tif')
            #la ground truth ha pixel 1 (no change) e 2 (change)
            gt = np.array(gt) - 1

            #reshape dell'immagine alle dimensioni WxHxC
            img1=np.array(img1_list).transpose((1,2,0))
            img2=np.array(img2_list).transpose((1,2,0))

            print(img1.shape)
            if self.normalize:
                           
              #print(img1.shape,img1.mean(axis=(0,1)))

              img1=(img1-img1.mean(axis=(0,1)))/img1.std(axis=(0,1))
              img2=(img2-img2.mean(axis=(0,1)))/img2.std(axis=(0,1))

              print(img1.shape,img1.mean(axis=(0,1)),img1.std(axis=(0,1)))

            img1=torch.from_numpy(img1).to(torch.float64)
            img2=torch.from_numpy(img2).to(torch.float64)
            gt=torch.from_numpy(gt)

            all_data.append((img1, img2, gt))
        return all_data

train_set_imgs=OSCD_dataset(train=True,transform=transform)

test_set_imgs=OSCD_dataset(train=False,transform=transform)

loading: aguasclaras
(525, 471, 13)
(525, 471, 13) [ 6.59351476e-16 -1.96776733e-16 -1.92179145e-16 -1.66662536e-17
  1.63214346e-16  2.55740789e-17  3.68381675e-16  1.62898262e-16
 -2.98785699e-16 -3.70105770e-16 -5.12171215e-16  7.08028430e-17
 -2.16776237e-16] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
loading: bercy
(360, 395, 13)
(360, 395, 13) [-8.99421185e-16 -4.69098116e-16 -4.05339147e-16 -1.45506361e-16
  3.59768474e-17 -2.23855939e-17 -1.15125912e-16 -2.55835359e-17
 -5.75629558e-17 -2.09465200e-16  1.92576069e-16  1.16724883e-16
  1.39909962e-17] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
loading: bordeaux
(461, 517, 13)
(461, 517, 13) [-5.58090436e-17  7.96590622e-17 -6.72570525e-17  9.62348251e-17
 -4.23934081e-17  1.43100112e-16 -1.45485114e-16 -7.39350577e-17
 -1.21873595e-16  1.31175102e-16 -1.69454382e-16  1.60272125e-16
  4.25126582e-17] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
loading: nantes


KeyboardInterrupt: ignored

In [11]:
train_loader = DataLoader(train_set_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set_imgs, batch_size=32, shuffle=True)

print()

In [35]:
for batch_idx, (data, target) in enumerate(train_loader):
  print(data,target)

<ipython-input-10-a01eee033482>:51: FutureWarning: The input object of type 'TiffImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'TiffImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  img1_list=np.array(sample[0])
<ipython-input-10-a01eee033482>:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  img1_list=np.array(sample[0])


TypeError: ignored

In [ ]:
mean = torch.mean(all_images_tensor)
std = torch.std(all_images_tensor)


In [ ]:
a=torch.tensor([[1, 2],[3, 4]])
b=torch.tensor([[5, 6],[7, 8]])
c=torch.stack((a,b))
print(c)
print(c.shape)